# Use BulkWriter for Data Import (3): Use BulkImport

This notebook helps you learn how to use PyMilvus' BulkImport API to import your prepared data to a Zilliz Cloud collection.

## Before you start

Ensure that:

- Install the dependencies, including PyMilvus (2.2.16) and MinIO Python Client.
- Create an output folder for the storage of the BulkWriter output.

In [1]:
!pip install pymilvus==2.2.16 minio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.1/159.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.1 MB/s eta 0:00:00


# Import dependencies



In [14]:
from pathlib import Path
from urllib.parse import urlparse
import sys, time, json
import threading

import pandas as pd
from minio import Minio

from pymilvus import (
    connections,
    FieldSchema, CollectionSchema, DataType,
    Collection,
    utility,
    LocalBulkWriter,
    RemoteBulkWriter,
    BulkFileType,
    BulkInsertState,
    bulk_import,
    get_import_progress,
    list_import_jobs,
)

## Check prepared data

Once you have prepared your data using a LocalBulkWriter and upload the generated file to your object storage, or prepared your data using a RemoteBulkWriter and got the path to the remote folder. You are ready to import them to a Zilliz Cloud collection.

To check whether they are ready, do as follows:

In [15]:
YOUR_OBJECT_STORAGE_ACCESS_KEY = ""
YOUR_OBJECT_STORAGE_SECRET_KEY = ""
YOUR_OBJECT_STORAGE_BUCKET_NAME = ""
PATH_TO_YOUR_PREPARED_DATA_FILES_FOLDER = ""

client = Minio(
    endpoint="s3.amazonaws.com", # use 'storage.googleapis.com' for GCS
    access_key=YOUR_OBJECT_STORAGE_ACCESS_KEY,
    secret_key=YOUR_OBJECT_STORAGE_SECRET_KEY,
    secure=True)

objects = client.list_objects(
    bucket_name=YOUR_OBJECT_STORAGE_BUCKET_NAME,
    prefix=PATH_TO_YOUR_PREPARED_DATA_FILES_FOLDER,
    recursive=True
)

for obj in objects:
    print(obj.object_name)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): s3.us-east-1.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://s3.us-east-1.amazonaws.com:443 "GET /doc-demo-1?location= HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): doc-demo-1.s3.us-west-2.amazonaws.com:443
DEBUG:urllib3.connectionpool:https://doc-demo-1.s3.us-west-2.amazonaws.com:443 "GET /?delimiter=&encoding-type=url&list-type=2&max-keys=1000&prefix=medium_articles%2Fa1f14844-1722-47e1-990e-2b4a735e1ed2 HTTP/1.1" 200 None


medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/claps.npy
medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/id.npy
medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/link.npy
medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/publication.npy
medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/reading_time.npy
medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/responses.npy
medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/title.npy
medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/vector.npy


## Create collection and import data

Once your data files are ready, connect to a Zilliz Cloud cluster, create a collection out of the schema, and import the data from the files in the storage bucket.

Since Zilliz Cloud does not allow cross-cloud data transmission, you need to create your cluster on the same public cloud that house your prepared dataset.

In [16]:
YOUR_CLUSTER_ENDPOINT = ""
YOUR_CLUSTER_TOKEN = ""
YOUR_COLLECTION_NAME = ""
YOUR_API_KEY = ""
CLUSTER_ID = urlparse(YOUR_CLUSTER_ENDPOINT).netloc.split(".")[0] if urlparse(YOUR_CLUSTER_ENDPOINT).netloc.startswith("in") else None
CLOUD_REGION = urlparse(YOUR_CLUSTER_ENDPOINT).netloc.split(".")[1] if urlparse(YOUR_CLUSTER_ENDPOINT).netloc.startswith("in") else 'aws-us-west-2'
OBJECT_URL = f"s3://{YOUR_OBJECT_STORAGE_BUCKET_NAME}/{PATH_TO_YOUR_PREPARED_DATA_FILES_FOLDER}/"

fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=768),
    FieldSchema(name="link", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="reading_time", dtype=DataType.INT64),
    FieldSchema(name="publication", dtype=DataType.VARCHAR, max_length=512),
    FieldSchema(name="claps", dtype=DataType.INT64),
    FieldSchema(name="responses", dtype=DataType.INT64)
]

schema = CollectionSchema(fields)

connections.connect(
    uri=YOUR_CLUSTER_ENDPOINT,
    token=YOUR_CLUSTER_TOKEN,
    secure=True
)

collection = Collection(YOUR_COLLECTION_NAME, schema)

collection.create_index(
    field_name="vector",
    index_params={
        "index_type": "AUTOINDEX",
        "metric_type": "L2"
    }
)

collection.load()

res = bulk_import(
    url=f"controller.api.{CLOUD_REGION}.zillizcloud.com",
    api_key=YOUR_API_KEY,
    object_url=OBJECT_URL,
    access_key=YOUR_OBJECT_STORAGE_ACCESS_KEY,
    secret_key=YOUR_OBJECT_STORAGE_SECRET_KEY,
    cluster_id=CLUSTER_ID,
    collection_name=YOUR_COLLECTION_NAME
)

print(json.dumps(res.json(), indent=4))

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): controller.api.aws-us-west-2.zillizcloud.com:443
DEBUG:urllib3.connectionpool:https://controller.api.aws-us-west-2.zillizcloud.com:443 "POST /v1/vector/collections/import HTTP/1.1" 200 69


{
    "code": 200,
    "data": {
        "jobId": "6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179"
    }
}


## Check Bulk Import Progress

You can check the progress of a specified bulk-import job.

In [19]:
job_id = res.json()['data']['jobId']
res = get_import_progress(
    url=f"controller.api.{CLOUD_REGION}.zillizcloud.com",
    api_key=YOUR_API_KEY,
    job_id=job_id,
    cluster_id=CLUSTER_ID
)

while res.json()["data"]["readyPercentage"] < 1:
    time.sleep(5)
    print(res.json())

    res = get_import_progress(
        url=f"controller.api.{CLOUD_REGION}.zillizcloud.com",
        api_key=YOUR_API_KEY,
        job_id=job_id,
        cluster_id=CLUSTER_ID
    )

print(json.dumps(res.json(), indent=4))

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): controller.api.aws-us-west-2.zillizcloud.com:443
DEBUG:urllib3.connectionpool:https://controller.api.aws-us-west-2.zillizcloud.com:443 "GET /v1/vector/collections/import/get?jobId=6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179&clusterId=in01-55aa41ad635aafc HTTP/1.1" 200 420
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): controller.api.aws-us-west-2.zillizcloud.com:443


{'code': 200, 'data': {'collectionName': 'medium_articles', 'fileName': 'medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/', 'fileSize': 26571700, 'readyPercentage': 0.5, 'completeTime': None, 'errorMessage': None, 'jobId': '6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179', 'details': [{'fileName': 'medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/', 'fileSize': 26571700, 'readyPercentage': 0.5, 'completeTime': None, 'errorMessage': None}]}}


DEBUG:urllib3.connectionpool:https://controller.api.aws-us-west-2.zillizcloud.com:443 "GET /v1/vector/collections/import/get?jobId=6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179&clusterId=in01-55aa41ad635aafc HTTP/1.1" 200 420
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): controller.api.aws-us-west-2.zillizcloud.com:443


{'code': 200, 'data': {'collectionName': 'medium_articles', 'fileName': 'medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/', 'fileSize': 26571700, 'readyPercentage': 0.5, 'completeTime': None, 'errorMessage': None, 'jobId': '6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179', 'details': [{'fileName': 'medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/', 'fileSize': 26571700, 'readyPercentage': 0.5, 'completeTime': None, 'errorMessage': None}]}}


DEBUG:urllib3.connectionpool:https://controller.api.aws-us-west-2.zillizcloud.com:443 "GET /v1/vector/collections/import/get?jobId=6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179&clusterId=in01-55aa41ad635aafc HTTP/1.1" 200 420
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): controller.api.aws-us-west-2.zillizcloud.com:443


...

{'code': 200, 'data': {'collectionName': 'medium_articles', 'fileName': 'medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/', 'fileSize': 26571700, 'readyPercentage': 0.9, 'completeTime': None, 'errorMessage': None, 'jobId': '6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179', 'details': [{'fileName': 'medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/', 'fileSize': 26571700, 'readyPercentage': 0.9, 'completeTime': None, 'errorMessage': None}]}}


DEBUG:urllib3.connectionpool:https://controller.api.aws-us-west-2.zillizcloud.com:443 "GET /v1/vector/collections/import/get?jobId=6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179&clusterId=in01-55aa41ad635aafc HTTP/1.1" 200 420
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): controller.api.aws-us-west-2.zillizcloud.com:443


{'code': 200, 'data': {'collectionName': 'medium_articles', 'fileName': 'medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/', 'fileSize': 26571700, 'readyPercentage': 0.9, 'completeTime': None, 'errorMessage': None, 'jobId': '6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179', 'details': [{'fileName': 'medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/', 'fileSize': 26571700, 'readyPercentage': 0.9, 'completeTime': None, 'errorMessage': None}]}}


DEBUG:urllib3.connectionpool:https://controller.api.aws-us-west-2.zillizcloud.com:443 "GET /v1/vector/collections/import/get?jobId=6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179&clusterId=in01-55aa41ad635aafc HTTP/1.1" 200 452


{
    "code": 200,
    "data": {
        "collectionName": "medium_articles",
        "fileName": "medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/",
        "fileSize": 26571700,
        "readyPercentage": 1,
        "completeTime": "2023-09-18T11:10:59Z",
        "errorMessage": null,
        "jobId": "6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179",
        "details": [
            {
                "fileName": "medium_articles/a1f14844-1722-47e1-990e-2b4a735e1ed2/1/",
                "fileSize": 26571700,
                "readyPercentage": 1,
                "completeTime": "2023-09-18T11:10:59Z",
                "errorMessage": null
            }
        ]
    }
}


## List all bulk-import jobs

You can list all bulk-import jobs to learn about their details.

In [20]:
res = list_import_jobs(
    url=f"controller.api.{CLOUD_REGION}.zillizcloud.com",
    api_key=YOUR_API_KEY,
    cluster_id=CLUSTER_ID,
    page_size=10,
    current_page=1,
)

print(json.dumps(res.json(), indent=4))

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): controller.api.aws-us-west-2.zillizcloud.com:443
DEBUG:urllib3.connectionpool:https://controller.api.aws-us-west-2.zillizcloud.com:443 "GET /v1/vector/collections/import/list?clusterId=in01-55aa41ad635aafc&pageSize=10&currentPage=1 HTTP/1.1" 200 632


{
    "code": 200,
    "data": {
        "tasks": [
            {
                "collectionName": "medium_articles",
                "jobId": "6e1d3f1a-3ff7-4d1d-9b8c-a7513a5e2179",
                "state": "ImportCompleted"
            },
            {
                "collectionName": "medium_articles",
                "jobId": "7371b114-6510-4135-ae92-7edebde7ea0a",
                "state": "ImportCompleted"
            },
            {
                "collectionName": "medium_articles",
                "jobId": "32bb12a5-ee49-4466-af33-11ed7d35b102",
                "state": "ImportCompleted"
            },
            {
                "collectionName": "medium_articles_2020",
                "jobId": "05eae2a4-9127-4ddd-a6ee-fb2303fdd115",
                "state": "ImportCompleted"
            },
            {
                "collectionName": "medium_articles_2020",
                "jobId": "9502308c-31a0-4c7e-a74d-f4b42014af57",
                "state": "ImportCompleted"
   